In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler,MinMaxScaler

In [2]:
for file in ["train", "test"]:

    #obtain some extra info
    df = pd.read_csv(file+"_features.csv")

    ratios = pd.DataFrame(df.groupby(["pid"]).last()/df.groupby(["pid"]).first()-1).iloc[:,2:]
    ratios.columns = [x+"_ratio" for x in ratios.columns]
    ratios.reset_index(drop=True, inplace=True)

    derivative = pd.DataFrame(df.groupby(["pid"]).last()-df.groupby(["pid"]).first()).iloc[:,2:]
    derivative.columns = [x+"_derivative" for x in derivative.columns]
    derivative.reset_index(drop=True, inplace=True)

    count = pd.DataFrame(df.groupby(["pid"]).count()).iloc[:,2:]
    count.columns = [x+"_count" for x in count.columns]
    count.reset_index(drop=True, inplace=True)

    #which values are missing
    columns = ["isnan_"+str(x) for x in range(1,409)]
    data = np.array(df)[:, 3:]
    data = data.reshape(data.shape[0]//12,34*12)
    missingval = pd.DataFrame(data, columns=columns)
    missingval = missingval.fillna(-9000)
    missingval = (missingval==-9000).astype(int)

    #total number of nan values
    total_nan = missingval.sum(axis=1)


    df = pd.read_csv(file+"_features.csv",index_col="pid")
    #forward fill missing values
    df = df.groupby(["pid"]).ffill().reset_index()
    df.set_index("pid",inplace=True)
    #backward fill missing values
    df = df.groupby(["pid"]).bfill().reset_index()

    # df = pd.read_csv(file+"_features.csv",index_col="pid").reset_index()

    #some columns are missing entirely, create new columns to indicate that those values were imputed
    # missing = df.groupby(["pid"]).last().isna().astype(int).iloc[:,2:]
    # missing.columns = [x+"_missing" for x in missing.columns]
    # missing.reset_index(drop=True, inplace=True)

    #group every 12 rows together
    data = np.array(df)[:, 3:]
    data = data.reshape(data.shape[0]//12,34*12)
    data = pd.DataFrame(data, columns=np.concatenate([[col+"_"+str(x) for x in range(1,13)] for col in df.columns[3:]]))
    # data = pd.DataFrame(data, columns=["data_"+str(x) for x in range(1,409)])

    #normalize data
    # scaler = MinMaxScaler()
    # df_scaled = scaler.fit_transform(data)
    # df_scaled = pd.DataFrame(df_scaled, columns=["scaled_data_"+str(x) for x in range(1,409)])

    #append
    df1 = df.iloc[::12,:3]
    df2 = data
    df1.reset_index(drop=True, inplace=True)
    df2.reset_index(drop=True, inplace=True)
    # df = pd.concat([df1,df2,df_scaled,missing,ratios,derivative,count,missingval,total_nan],axis=1)
    df = pd.concat([df1,df2,ratios,derivative,count,total_nan],axis=1)
    #fill nan
    df = df.fillna(0)
    #replace inf with 0
    df = df.replace([np.inf, -np.inf], 0)

    df.to_csv(file+'_features_improved.csv', index=False, float_format='%.3f',na_rep='nan')



In [3]:
df = pd.read_csv("train_features_improved.csv")
df.head()

,pid,Time,Age,EtCO2_1,EtCO2_2,EtCO2_3,EtCO2_4,EtCO2_5,EtCO2_6,EtCO2_7,...,SpO2_count,Bilirubin_direct_count,Chloride_count,Hct_count,Heartrate_count,Bilirubin_total_count,TroponinI_count,ABPs_count,pH_count,0
0,1,3,34.0,0.0,0.0,12.0,0.00,36.0,8.7,24.0,...,12,0,3,6,12,0,0,12,7,274
1,10,1,71.0,0.0,27.8,12.0,0.00,36.0,14.6,0.0,...,11,0,0,1,11,0,1,11,0,324
2,100,2,68.0,0.0,20.9,21.0,0.00,35.0,12.5,27.0,...,11,1,0,2,11,1,1,11,0,311
3,1000,1,79.0,26.0,0.0,22.0,4.05,36.0,9.2,0.0,...,12,0,2,10,12,0,0,12,7,297
4,10000,1,76.0,0.0,25.7,22.0,0.00,36.0,11.0,25.0,...,10,0,0,0,11,0,1,11,0,294


In [4]:
#other predictions
# df = pd.read_csv("submit.csv")
# df.iloc[:,:11]